In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from circadian.models import Skeldon23
from circadian.lights import LightSchedule
from utils.helper_functions import H_thresholds

In [2]:
save_path = 'data/different_initial_conditions'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
light_data_location = 'data/light_schedules/'

dt = 0.005 # hours
days = 14.0
regular_indoor_lux = 500.0
time = np.arange(0, 24*days, dt)

parameters = {
    'mu': 19.0,
    'Delta': 6.0,
    'chi': 11.0,
    'k': 0.55,
    'p': 0.6,
    'tauc': 24.2,
}

lights_on_start_times = [4, 5, 6, 7, 8, 9]
regular_light_options = []


for lights_on in lights_on_start_times:
    regular_light = LightSchedule.Regular(regular_indoor_lux, lights_on=lights_on, lights_off=int(np.mod(lights_on + 16, 24)))
    regular_light_options.append(regular_light(time))

In [9]:
forced_wakeup_threshold = 0.5 # lux

chang14_ebook_first_light = np.load(f"{light_data_location}/chang14_ebook_first_light.npy")
chang14_ebook_second_light = np.load(f"{light_data_location}/chang14_ebook_second_light.npy")
chang14_high_ebook_first_light = np.load(f"{light_data_location}/chang14_high_ebook_first_light.npy")
chang14_high_ebook_second_light = np.load(f"{light_data_location}/chang14_high_ebook_second_light.npy")

results = {}

idx = 0
for regular_light in tqdm(regular_light_options):
    start_time = lights_on_start_times[idx]
    idx += 1
    # Equilibrate
    mu = parameters['mu']
    delta = parameters['Delta']
    chi = parameters['chi']
    p = parameters['p']
    k = parameters['k']
    tauc = parameters['tauc']

    parameters = {'S0': 1.0, 'mu': mu, 'Delta': delta, 'chi': chi, 'p': p, 'k': k, 'tauc': tauc, 
                  'forced_wakeup_light_threshold': forced_wakeup_threshold,}
    equilibration_model = Skeldon23(params=parameters)
    initial_condition = equilibration_model.equilibrate(time, regular_light, num_loops=10)

    # Simulate
    parameters = {
        'S0': 1.0,
        'mu': mu, 'Delta': delta, 'chi': chi,
        'p': p, 'k': k, 'tauc': tauc,
        'forced_wakeup_light_threshold': forced_wakeup_threshold,
    }
    # Chang14 Ebook first simulation
    chang14_ebook_first_model = Skeldon23(params=parameters)
    chang14_ebook_first_trajectory = chang14_ebook_first_model.integrate(time, 
                                                                        initial_condition=initial_condition, 
                                                                        input=chang14_ebook_first_light)
    
    # Chang14 Ebook second simulation
    chang14_ebook_second_model = Skeldon23(params=parameters)
    chang14_ebook_second_trajectory = chang14_ebook_second_model.integrate(time, 
                                                        initial_condition=initial_condition,
                                                        input=chang14_ebook_second_light)
    chang14_ebook_second_H_plus, chang14_ebook_second_H_minus = H_thresholds(chang14_ebook_second_model, chang14_ebook_second_trajectory)

    ebook_first_cbtmin = chang14_ebook_first_model.cbt(chang14_ebook_first_trajectory)[:-1]
    ebook_second_cbtmin = chang14_ebook_second_model.cbt(chang14_ebook_second_trajectory)[:-1]

    # Chang14 High Ebook first simulation
    chang14_high_ebook_first_model = Skeldon23(params=parameters)
    chang14_high_ebook_first_trajectory = chang14_high_ebook_first_model.integrate(time,
                                                                                initial_condition=initial_condition,
                                                                                input=chang14_high_ebook_first_light)
    chang14_high_ebook_first_H_plus, chang14_high_ebook_first_H_minus = H_thresholds(chang14_high_ebook_first_model, chang14_high_ebook_first_trajectory)
    # Chang14 High Ebook second simulation
    chang14_high_ebook_second_model = Skeldon23(params=parameters)
    chang14_high_ebook_second_trajectory = chang14_high_ebook_second_model.integrate(time,
                                                                                initial_condition=initial_condition,
                                                                                input=chang14_high_ebook_second_light)

    # Save data
    np.savez(f"{save_path}/different_initial_conditions_{start_time}_lights_on.npz",
            time=time,
            parameters=parameters,
            initial_condition=initial_condition,
            regular_light=regular_light,
            regular_sleep=equilibration_model.sleep_state,
            chang14_ebook_first_trajectory_states=chang14_ebook_first_trajectory.states,
            chang14_ebook_first_light=chang14_ebook_first_light,
            chang14_ebook_first_sleep=chang14_ebook_first_model.sleep_state,
            chang14_ebook_second_trajectory_states=chang14_ebook_second_trajectory.states,
            chang14_ebook_second_light=chang14_ebook_second_light,
            chang14_ebook_second_sleep=chang14_ebook_second_model.sleep_state,
            chang14_ebook_second_H_plus=chang14_ebook_second_H_plus,
            chang14_ebook_second_H_minus=chang14_ebook_second_H_minus,
            chang14_high_ebook_first_trajectory_states=chang14_high_ebook_first_trajectory.states,
            chang14_high_ebook_first_light=chang14_high_ebook_first_light,
            chang14_high_ebook_first_sleep=chang14_high_ebook_first_model.sleep_state,
            chang14_high_ebook_first_H_plus=chang14_high_ebook_first_H_plus,
            chang14_high_ebook_first_H_minus=chang14_high_ebook_first_H_minus,
            chang14_high_ebook_second_trajectory_states=chang14_high_ebook_second_trajectory.states,
            chang14_high_ebook_second_light=chang14_high_ebook_second_light,
            chang14_high_ebook_second_sleep=chang14_high_ebook_second_model.sleep_state,
    )

  0%|          | 0/1 [00:00<?, ?it/s]